<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#&quot;Дерево-решений&quot;.-Циклом-ищем-лучший-гиперпораметр-max_depth" data-toc-modified-id="&quot;Дерево-решений&quot;.-Циклом-ищем-лучший-гиперпораметр-max_depth-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>"Дерево решений". Циклом ищем лучший гиперпораметр max_depth</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    recall_score, 
    precision_score, 
    roc_auc_score, 
    roc_curve, 
    confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import RobustScaler

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Для последующего обучения не информативными являются столбцы "RowNumber"-порядковый номер, "Surname"- фамилия. Данные столбцы подлежат удалению.

In [4]:
for_drop = ['RowNumber', 'Surname', 'CustomerId']
df_good = df.drop(for_drop, axis=1)
df_good.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
df_good = df_good.rename(columns={'CreditScore':'credit_score',
                                  'NumOfProducts':'num_of_products',
                                  'HasCrCard':'has_cr_card',
                                  'IsActiveMember':'is_active_member',
                                  'EstimatedSalary': 'estimated_salary'
                                 })
df_good.head()

,credit_score,Geography,Gender,Age,Tenure,Balance,num_of_products,has_cr_card,is_active_member,estimated_salary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
df_good['Tenure'].isna().sum()

909

In [7]:
df_good['Tenure'].fillna(df_good['Tenure'].median(), inplace=True)

In [8]:
df_good['Tenure'] = df_good['Tenure'].astype('int')

In [9]:
df_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   credit_score      10000 non-null  int64  
 1   Geography         10000 non-null  object 
 2   Gender            10000 non-null  object 
 3   Age               10000 non-null  int64  
 4   Tenure            10000 non-null  int64  
 5   Balance           10000 non-null  float64
 6   num_of_products   10000 non-null  int64  
 7   has_cr_card       10000 non-null  int64  
 8   is_active_member  10000 non-null  int64  
 9   estimated_salary  10000 non-null  float64
 10  Exited            10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


Анализ показал, что столбец "Tenure" содержит пропуски. Мы их заполнили медианным значением, с целью минимизации ошибки в дальнейшем обучении модели.

In [10]:
df.duplicated().sum()

0

## Исследование задачи

Подготовим данные для дальнейшего обучения

In [11]:
df_good = pd.get_dummies(df_good, drop_first=True)
df_good.head()

,credit_score,Age,Tenure,Balance,num_of_products,has_cr_card,is_active_member,estimated_salary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


В новом датафрейме видно, что столбец Geography содержал 3 класса, Gender_Male 2 класса 
Далее необходимо сформировать обучающую, валидационную и тестовую выборки

In [12]:
#создаю переменные для признаков и целевого признака:
features = df_good.drop(['Exited'], axis=1)
target = df_good['Exited']

#разделяю данные на обучающую и валидационную выборки в соотношении 60/40:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target, test_size=0.4,
                                                                              random_state=12345)

#разделяю валидационную выборку на валидационную и тестовую в соотношении 20/20:
features_test, features_valid, target_test, target_valid = train_test_split(features_valid,
                                                                            target_valid, test_size=0.5,
                                                                            random_state=12345)
print('Тренировочная:', features_train.shape)
print('Валидационная:', features_valid.shape)
print('Тестовая:', features_test.shape)


Тренировочная: (6000, 11)
Валидационная: (2000, 11)
Тестовая: (2000, 11)


In [13]:
numeric = ['credit_score', 'Age', 'Tenure', 'Balance', 'num_of_products', 'estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

In [14]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

После того, как мы получили масштабированные признаки для трех выборок, перейдем к обучению моделей.

In [15]:
for depth in range(1,15,1):
    model_tree=DecisionTreeClassifier(max_depth=depth,random_state=1234).fit(features_train,target_train)
    prediction=model_tree.predict(features_valid)
    print('max_depth:',depth,'F1:',f1_score(target_valid,prediction))

max_depth: 1 F1: 0.0
max_depth: 2 F1: 0.4986301369863014
max_depth: 3 F1: 0.3795620437956204
max_depth: 4 F1: 0.48307692307692307
max_depth: 5 F1: 0.5015772870662459
max_depth: 6 F1: 0.5253456221198156
max_depth: 7 F1: 0.5045592705167173
max_depth: 8 F1: 0.4976958525345621
max_depth: 9 F1: 0.5263157894736842
max_depth: 10 F1: 0.4999999999999999
max_depth: 11 F1: 0.5067024128686327
max_depth: 12 F1: 0.4869109947643979
max_depth: 13 F1: 0.4869791666666667
max_depth: 14 F1: 0.48415716096324457


In [16]:
probabilities_tree=model_tree.predict_proba(features_valid)
probabilities_one_valid_tree=probabilities_tree[:,1]
fpr_tree,tpr_tree,thresholds=roc_curve(target_valid,probabilities_one_valid_tree)
auc_roc_tree=roc_auc_score(target_valid,probabilities_one_valid_tree)
print('AUC-ROC =', auc_roc_tree)

AUC-ROC = 0.6883802473799641


Обучение с помощью "дерева решений" показало лучшее значение max_depth=9, при котором F1=0,5263. AUC-ROC = 0.6884. Далее, обучение методом "случайный лес"

In [17]:
for max_depth in range(1,25,1):
    model_forest=RandomForestClassifier(max_depth=max_depth,n_estimators=50,random_state=1234).fit(features_train,target_train)
    prediction=model_forest.predict(features_valid)
    print('max_depth:',max_depth,'F1:',f1_score(target_valid,prediction))

max_depth: 1 F1: 0.0
max_depth: 2 F1: 0.16738197424892703
max_depth: 3 F1: 0.21487603305785122
max_depth: 4 F1: 0.3005780346820809
max_depth: 5 F1: 0.41166380789022305
max_depth: 6 F1: 0.47019867549668865
max_depth: 7 F1: 0.4727272727272727
max_depth: 8 F1: 0.48309178743961356
max_depth: 9 F1: 0.49201277955271566
max_depth: 10 F1: 0.5040128410914929
max_depth: 11 F1: 0.5046728971962616
max_depth: 12 F1: 0.5141065830721003
max_depth: 13 F1: 0.5390505359877489
max_depth: 14 F1: 0.5261538461538461
max_depth: 15 F1: 0.542728635682159
max_depth: 16 F1: 0.5421686746987953
max_depth: 17 F1: 0.5421686746987953
max_depth: 18 F1: 0.540785498489426
max_depth: 19 F1: 0.5303030303030303
max_depth: 20 F1: 0.5248868778280542
max_depth: 21 F1: 0.5385779122541604
max_depth: 22 F1: 0.5319148936170214
max_depth: 23 F1: 0.5245398773006135
max_depth: 24 F1: 0.5298621745788668


In [18]:
for estim in range(1,55,2):
    model_forest=RandomForestClassifier(max_depth=15,n_estimators=estim,random_state=1234).fit(features_train,target_train)
    prediction=model_forest.predict(features_valid)
    print('estim',estim,'F1:',f1_score(target_valid,prediction))

estim 1 F1: 0.47655259822560203
estim 3 F1: 0.5
estim 5 F1: 0.5165945165945166
estim 7 F1: 0.5087719298245613
estim 9 F1: 0.5387994143484627
estim 11 F1: 0.5291479820627802
estim 13 F1: 0.5398230088495576
estim 15 F1: 0.5244444444444445
estim 17 F1: 0.5209580838323352
estim 19 F1: 0.5233082706766917
estim 21 F1: 0.5189681335356601
estim 23 F1: 0.5339366515837104
estim 25 F1: 0.532724505327245
estim 27 F1: 0.5317220543806647
estim 29 F1: 0.5266362252663622
estim 31 F1: 0.5256797583081572
estim 33 F1: 0.5279034690799397
estim 35 F1: 0.5365126676602088
estim 37 F1: 0.5293233082706768
estim 39 F1: 0.5389221556886228
estim 41 F1: 0.5367316341829085
estim 43 F1: 0.5397301349325337
estim 45 F1: 0.5373134328358209
estim 47 F1: 0.5468053491827638
estim 49 F1: 0.5462686567164179
estim 51 F1: 0.5386904761904763
estim 53 F1: 0.5389221556886228


In [19]:
probabilities_forest=model_forest.predict_proba(features_valid)
probabilities_one_valid_forest=probabilities_forest[:,1]
fpr_forest,tpr_forest,thresholds=roc_curve(target_valid,probabilities_one_valid_forest)
auc_roc_forest=roc_auc_score(target_valid,probabilities_one_valid_forest)
print('AUC-ROC =', auc_roc_forest)

AUC-ROC = 0.8419665972587626


F1 принимает максимальное значение для модели "случайный лес" с параметрами max_depth=15 и estim 47 F1= 0.5468. AUC-ROC = 0.8419


Третий метод- логистическая регрессия

In [20]:
model_logistic=LogisticRegression().fit(features_train,target_train)
prediction=model_logistic.predict(features_valid)
print('F1:',f1_score(target_valid,prediction))

F1: 0.27177700348432055


In [21]:
probabilities_reg=model_logistic.predict_proba(features_valid)
probabilities_one_valid_reg=probabilities_reg[:,1]
fpr_log,tpr_log,thresholds=roc_curve(target_valid,probabilities_one_valid_reg)
auc_roc_reg=roc_auc_score(target_valid,probabilities_one_valid_reg)
print('AUC-ROC =', auc_roc_reg)

AUC-ROC = 0.7387189669465469


F1 = 0.2717, наименьшее значение среди всех моделей, также как и AUC-ROC = 0.7387

Без учёта дисбаланса наибольшее значение F1 наблюдается в моделях дерево решений и случайный лес - 0.5468 и 0.5263 соответственно.

## Борьба с дисбалансом

Так как несбалансированность классов напрямую влияет на точность и качество результатов машинного обучения, необходимо ее устранить, увеличив число наблюдений положительного класса

In [22]:

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

## "Дерево решений". Циклом ищем лучший гиперпораметр max_depth

In [23]:
%%time

for depth in range(1,75,1):
    model_tree=DecisionTreeClassifier(class_weight='balanced',max_depth=depth,random_state=1234).fit(features_upsampled,target_upsampled)
  
    prediction=model_tree.predict(features_valid)
    print('max_depth:',depth,'F1:',f1_score(target_valid,prediction))

max_depth: 1 F1: 0.48514851485148514
max_depth: 2 F1: 0.5129682997118156
max_depth: 3 F1: 0.5129682997118156
max_depth: 4 F1: 0.5385239253852393
max_depth: 5 F1: 0.5809128630705395
max_depth: 6 F1: 0.5643835616438356
max_depth: 7 F1: 0.5618408437200384
max_depth: 8 F1: 0.550046772684752
max_depth: 9 F1: 0.5428015564202334
max_depth: 10 F1: 0.5254074784276127
max_depth: 11 F1: 0.5173824130879346
max_depth: 12 F1: 0.531540847983454
max_depth: 13 F1: 0.5222101841820151
max_depth: 14 F1: 0.5237020316027088
max_depth: 15 F1: 0.5051903114186851
max_depth: 16 F1: 0.5011709601873535
max_depth: 17 F1: 0.5047393364928909
max_depth: 18 F1: 0.5
max_depth: 19 F1: 0.4993849938499385
max_depth: 20 F1: 0.47466007416563655
max_depth: 21 F1: 0.4814814814814815
max_depth: 22 F1: 0.4920828258221681
max_depth: 23 F1: 0.47735618115055084
max_depth: 24 F1: 0.47735618115055084
max_depth: 25 F1: 0.47735618115055084
max_depth: 26 F1: 0.47735618115055084
max_depth: 27 F1: 0.47735618115055084
max_depth: 28 F1: 0.

In [24]:
probabilities_tree=model_tree.predict_proba(features_valid)
probabilities_one_valid_tree=probabilities_tree[:,1]
fpr_tree,tpr_tree,thresholds=roc_curve(target_valid,probabilities_one_valid_tree)
auc_roc_tree=roc_auc_score(target_valid,probabilities_one_valid_tree)
print('AUC-ROC =', auc_roc_tree)

AUC-ROC = 0.6674019707047676


При увеличении выборки для "дерева решений" параметр F1 удалось увеличить до 0,5809, значение AUC-ROC равно 0.6718

In [25]:
%%time

for max_depth in range(1,20,1):
    model_forest=RandomForestClassifier(class_weight='balanced',max_depth=max_depth,n_estimators=50,random_state=1234).fit(features_upsampled,target_upsampled)
    prediction=model_forest.predict(features_valid)
    print('max_depth:',max_depth,'F1:',f1_score(target_valid,prediction))

max_depth: 1 F1: 0.5383211678832117
max_depth: 2 F1: 0.5547169811320756
max_depth: 3 F1: 0.5700197238658776
max_depth: 4 F1: 0.5802816901408451
max_depth: 5 F1: 0.5971014492753624
max_depth: 6 F1: 0.6053412462908012
max_depth: 7 F1: 0.6007984031936128
max_depth: 8 F1: 0.612410986775178
max_depth: 9 F1: 0.6091476091476091
max_depth: 10 F1: 0.6143931256713211
max_depth: 11 F1: 0.6089813800657174
max_depth: 12 F1: 0.6002317497103128
max_depth: 13 F1: 0.5992779783393501
max_depth: 14 F1: 0.6019900497512438
max_depth: 15 F1: 0.5725288831835685
max_depth: 16 F1: 0.583547557840617
max_depth: 17 F1: 0.5795601552393272
max_depth: 18 F1: 0.5846560846560847
max_depth: 19 F1: 0.5802139037433155
CPU times: user 6.58 s, sys: 31 ms, total: 6.61 s
Wall time: 6.64 s


In [ ]:
%%time

for estim in range(1,51,1):
    model_forest=RandomForestClassifier(class_weight='balanced',max_depth=9,n_estimators=estim,random_state=1234).fit(features_upsampled,target_upsampled)
    prediction=model_forest.predict(features_valid)
    print('estim',estim,'F1:',f1_score(target_valid,prediction))

estim 1 F1: 0.5378151260504203
estim 2 F1: 0.5696969696969697
estim 3 F1: 0.5906735751295336
estim 4 F1: 0.5924369747899159
estim 5 F1: 0.6018808777429466
estim 6 F1: 0.6099585062240663
estim 7 F1: 0.6084210526315789
estim 8 F1: 0.6044071353620146
estim 9 F1: 0.6218487394957983
estim 10 F1: 0.6204756980351603
estim 11 F1: 0.612668743509865
estim 12 F1: 0.6104166666666667
estim 13 F1: 0.6157068062827226
estim 14 F1: 0.608066184074457
estim 15 F1: 0.620618556701031
estim 16 F1: 0.6043613707165109
estim 17 F1: 0.6128364389233955
estim 18 F1: 0.608875128998968
estim 19 F1: 0.6149068322981367
estim 20 F1: 0.6161825726141079
estim 21 F1: 0.6093264248704664
estim 22 F1: 0.6208333333333332
estim 23 F1: 0.6169989506820567
estim 24 F1: 0.6165099268547545
estim 25 F1: 0.6096033402922756
estim 26 F1: 0.6104166666666667


In [ ]:
probabilities_forest=model_forest.predict_proba(features_valid)
probabilities_one_valid_forest=probabilities_forest[:,1]
fpr_forest,tpr_forest,thresholds=roc_curve(target_valid,probabilities_one_valid_forest)
auc_roc_forest=roc_auc_score(target_valid,probabilities_one_valid_forest)
print('AUC-ROC =', auc_roc_forest)

Наибольший показатель при  max_depth: 9 и estim 38 F1= 0.6247

Логистическая регрессия

In [ ]:
%%time

model_logistic=LogisticRegression(class_weight='balanced',solver='liblinear').fit(features_upsampled,target_upsampled)
prediction=model_logistic.predict(features_valid)
print('F1:',f1_score(target_valid,prediction))

In [ ]:
probabilities_reg=model_logistic.predict_proba(features_valid)
probabilities_one_valid_reg=probabilities_reg[:,1]
fpr_log,tpr_log,thresholds=roc_curve(target_valid,probabilities_one_valid_reg)
auc_roc_reg=roc_auc_score(target_valid,probabilities_one_valid_reg)
print('AUC-ROC =', auc_roc_reg)

Уменьшение выборки

In [ ]:
#попробую также сделать объекты частого класса не такими частыми:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

"Дерево решений"

In [ ]:
%%time

for depth in range(1,75,1):
    model_tree=DecisionTreeClassifier(class_weight='balanced',max_depth=depth,random_state=1234).fit(features_downsampled,target_downsampled)
  
    prediction=model_tree.predict(features_valid)
    print('max_depth:',depth,'F1:',f1_score(target_valid,prediction))

In [ ]:
probabilities_tree=model_tree.predict_proba(features_valid)
probabilities_one_valid_tree=probabilities_tree[:,1]
fpr_tree,tpr_tree,thresholds=roc_curve(target_valid,probabilities_one_valid_tree)
auc_roc_tree=roc_auc_score(target_valid,probabilities_one_valid_tree)
print('AUC-ROC =', auc_roc_tree)

Параметр F1 удалось увеличить еще на несколько %, при max_depth: 5 F1= 0.6093

"Случайный лес"

In [ ]:
%%time

for max_depth in range(1,20,1):
    model_forest=RandomForestClassifier(class_weight='balanced',max_depth=max_depth,n_estimators=50,random_state=1234).fit(features_downsampled,target_downsampled)
    prediction=model_forest.predict(features_valid)
    print('max_depth:',max_depth,'F1:',f1_score(target_valid,prediction))

In [ ]:
%%time

for estim in range(1,51,1):
    model_forest=RandomForestClassifier(class_weight='balanced',max_depth=6,n_estimators=estim,random_state=1234).fit(features_downsampled,target_downsampled)
    prediction=model_forest.predict(features_valid)
    print('estim',estim,'F1:',f1_score(target_valid,prediction))
    

In [ ]:
probabilities_forest=model_forest.predict_proba(features_valid)
probabilities_one_valid_forest=probabilities_forest[:,1]
fpr_forest,tpr_forest,thresholds=roc_curve(target_valid,probabilities_one_valid_forest)
auc_roc_forest=roc_auc_score(target_valid,probabilities_one_valid_forest)
print('AUC-ROC =', auc_roc_forest)

Уменьшение позволило достичь для случайного леса показателя F1= 0.6007 при max_depth: 6 estim 38. AUC-ROC = 0.8418

"Логистическая регрессия"

In [ ]:
%%time

model_logistic=LogisticRegression(class_weight='balanced',solver='liblinear').fit(features_downsampled,target_downsampled)
prediction=model_logistic.predict(features_valid)
print('F1:',f1_score(target_valid,prediction))

In [ ]:
probabilities_reg=model_logistic.predict_proba(features_valid)
probabilities_one_valid_reg=probabilities_reg[:,1]
fpr_log,tpr_log,thresholds=roc_curve(target_valid,probabilities_one_valid_reg)
auc_roc_reg=roc_auc_score(target_valid,probabilities_one_valid_reg)
print('AUC-ROC =', auc_roc_reg)

Хоть и удалось добиться небольшого увеличения показателя F1, он все равно не удовлетворяет условию задания , т.к. его значение менее 0,59, и составляет 0.4854 

Полученные значения F1 и AUC-ROC говорят о том, что наиболее подходящей моделью является "случайный лес" после устранения дисбаланса путем увеличения выборки. А именно AUC-ROC = 0.86011, F1= 0.6247

## Тестирование модели

Для итогового тестирования объединим выборки train и valid

In [ ]:
features_train_valid = pd.concat([features_train] + [features_valid])
target_train_valid = pd.concat([target_train] + [target_valid]) 
features_upsampled, target_upsampled = upsample(features_train_valid, target_train_valid, 4)



In [ ]:
model_rfc_test = RandomForestClassifier(class_weight='balanced', n_estimators=50, max_depth=6, random_state=12345)
model_rfc_test.fit(features_upsampled, target_upsampled)
predictions_rfc_test = model_rfc_test.predict(features_test)

print('F1 =', f1_score(target_test, predictions_rfc_test))
print('AUC-ROC =', roc_auc_score(target_test, model_rfc_test.predict_proba(features_test)[:,1]))


 

Чтобы добиться высокой точности и надежности модели была проделана следующая работа:
1. Предоставленные данные прошли предобработку, т.е. удалены столбцы с незначительными данными, преобразован тип данных, дубликаты удалены
2. Данные разделены на обучающую, валидационную выборки для дальнейшего обучения
3. Обучены модели методом "логистическая регрессия", "случайный лес" и "дерево решений", после чего мы избавились от дисбаланса путем увеличения и уменьшения выборок обученных методом "дерево решений" и "случайный лес", так как именно эти модели показали наибольшее значение параметра F1 и AUC-ROC
4. Проведено тестирование выбранной модели


Итоговое тестирование показало, что увеличение числа наблюдений положительного класса привело к увеличению точности модели (показатель F1 до равен 53% AUC-ROC = 84,91%, и после увеличения соответственно 58,1% И 84.93%)
Проверка на тестовой выборке подтвердила, что upsampling помог справиться с дисбалансом классов и удалось увеличить F1 до 60.72%, AUC-ROC до 84.1%.
Поскольку F1 - среднее гармоническое полноты и точности (растет только вместе с полнотой и точностью), то увеличение F1 свидетельствует об изменении в алгоритме в лучшую сторону. То есть вероятность ухода клиента из банка меньше вероятности того, что он останется.